In [ ]:
# Tracing (Langfuse)
# git clone https://github.com/langfuse/langfuse.git
# cd langfuse
# docker compose up
# test@test.com/test1234!

import os
from langfuse import get_client
 
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-0ef7a760-854c-4ca5-aba8-441372b21eeb" 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-380091d9-b6ab-4177-b9d5-854331cce9d1" 
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"
 
langfuse = get_client()

if langfuse.auth_check():
    from pydantic_ai import Agent
    Agent.instrument_all()
    print("Tracing is enabled.")
else:
    print("Authentication failed. Please check your credentials and host.")

In [ ]:
# Tracing (MLFlow)
# mlflow server --host 0.0.0.0 --port 5000

import mlflow

mlflow.pydantic_ai.autolog()
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Pydantic AI")
mlflow.tracing.disable_notebook_display()

In [ ]:
import os
import random
from dataclasses import dataclass
from dotenv import load_dotenv
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.google import GoogleModel, GoogleProvider
from pydantic_ai.settings import ModelSettings
from pydantic_ai.usage import UsageLimits

FREE_MODELS = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "gemma-3-27b-it",
    "gemma-3n-e4b-it",
    "gemini-embedding-001"
]

load_dotenv()
gemini_pro = GoogleModel(
    "gemini-2.5-pro",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemini_flash = GoogleModel(
    "gemini-2.5-flash",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemini_flash_lite = GoogleModel(
    "gemini-2.5-flash-lite-preview-06-17",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)
gemma = GoogleModel(
    "gemma-3-27b-it",
    provider=GoogleProvider(api_key=os.getenv("GEMINI_API_KEY")),
    settings=ModelSettings(temperature=0.8)
)

### Basic Functionality

In [ ]:
agent = Agent(gemini_flash_lite)
result = await agent.run("What is the capital of Italy?")
print(result.output)

In [ ]:
INSTRUCTIONS = """\
Answer the user's question using one of the tools provided."""

async def add_numbers(x: int, y: int) -> str:
    """
    Add two numbers together.

    Args:
        x: First number
        y: Second number
    """
    return str(x + y)

async def get_weather(city: str, state: str) -> str:
    """
    Get weather forecast for a location.

    Args:
        city: Name of the city
        state: State abbreviation (e.g., 'CA' for California)
    """
    return f"The weather in {city}, {state} is 75 degrees and sunny."

agent = Agent(
    gemini_flash_lite,
    instructions=INSTRUCTIONS,
    tools=[add_numbers, get_weather]
)

In [ ]:
result = await agent.run(
    "What's the weather like in Columbus, Ohio?",
    usage_limits=UsageLimits(request_limit=3)
)
print(result.output)

In [ ]:
result2 = await agent.run(
    "What about Dallas, Texas?",
    message_history=result.new_messages(),
    usage_limits=UsageLimits(request_limit=3)
)
print(result2.output)

### Streaming

In [ ]:
streamed_result = agent.run_stream(
    "What's the weather like in Columbus, Ohio?"
)
async with streamed_result as result:
    async for message in result.stream_text():
        print(message)

### Dependencies

In [ ]:
@dataclass
class User:
    name: str
    occupation: str

INSTRUCTIONS = """\
You're a dice game, you should roll the die and see if the number
you get back matches the user's guess. If so, tell them they're a winner.
Use the player's name in the response."""

def roll_dice() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))

def get_player_name(ctx: RunContext[User]) -> str:
    """Get the player's name."""
    return ctx.deps.name

agent = Agent(
    gemini_flash_lite,
    deps_type=User, 
    instructions=INSTRUCTIONS,
    tools=[roll_dice, get_player_name]
)

result = await agent.run("My guess is 4", deps=User("John", "Engineer"))
print(result.output)

### Structured Output

In [ ]:
INSTRUCTIONS = """\
Extract me the dimensions of a box.  If you can't extract all data, ask the user to try again."""

class Box(BaseModel):
    width: int
    height: int
    depth: int
    units: str

agent = Agent(
    gemini_flash_lite,
    output_type=[Box, str], 
    instructions=INSTRUCTIONS
)

result = await agent.run('The box is 10wx20hx30d cm')
print(result.output)

In [ ]:
async def format_response(box: Box) -> str:
    """
    Takes the extracted box dimensions and formats the output for the user/

    Args:
        box: Box object with extracted dimensions
    """
    return f"The box is this big: {str(box)}"

agent = Agent(
    gemini_flash_lite,
    output_type=[format_response, str], 
    instructions=INSTRUCTIONS
)

result = await agent.run('The box is 10wx20hx30d cm')
print(result.output)